## Backtesting
### Precio de Salida y Resultado de Órdenes de Trading
En este cuaderno se muestra como calcular si una órden abierta se cierra
con beneficio o pérdida según se establecen en el precio objetivo y el stop-loss


In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from mplfinance.original_flavor import candlestick2_ohlc
import mplfinance as mpf

_____
### Datos 

In [ ]:
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

In [ ]:
stock_df = stock_data['REE'].loc['2017':'2018']
stock_df.head()

_____

A partir de una orden de compra, que esperamos por precio objetivo o por stop-loss, 
tenemos que calcular **cuál de esos precios se alcanza primero**.
Si tenemos velas de precios a resolución aceptable, comparada con la amplitud de los precios objetivo y stop,  podemos considerar que:
- El precio de ejecución corresponde al precio del evento que ocurre primero
- Si tenemos una ventana temporal límite cerraríamos a precio final de la ventana

In [ ]:
windata = stock_df.iloc[0:70]
precio_objetivo = 14.75
precio_loss = 13.75

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.axhline(precio_objetivo, c='b')
ax.axhline(precio_loss, c='m')
mpf.plot(windata, ax=ax, type='candle', style='yahoo')

In [ ]:
over_prices = windata.high >= precio_objetivo
windata.high[over_prices]

In [ ]:
under_prices = windata.low <= precio_loss
windata.low[under_prices]

Bastaría con determinar por un lado si hay algun precio por encima o por debajo 
y en caso de ocurrir, saber cuál de los dos eventos ocurre primero

In [ ]:
def out_trade_bar(windata, p_target, p_loss):
    """Función que calcula para una ventana, la fecha y el precio de lo que ocurra primero.
      - Que se llegue al precio objetivo
      - Que se toque el precio stop
      - Que finalice la ventana del trade y se cierre la posicón
    """
    
    # si no hay ejecución asumimos salida al final de la ventana
    trade_date = windata.index[-1]
    trade_price = windata.close.iloc[-1]
    
    over_prices = windata.high >= p_target
    if over_prices.any():
        trade_date = windata.high[over_prices].index[0]
        trade_price = p_target

    under_prices = windata.low <= p_loss
    if under_prices.any() and windata.low[under_prices].index[0] < trade_date:
        trade_date = windata.low[under_prices].index[0]
        trade_price = p_loss

    return trade_date, trade_price

In [ ]:
out_trade_bar(windata, precio_objetivo, precio_loss)

In [ ]:
out_trade_bar(windata, precio_objetivo, 13.00)

In [ ]:
out_trade_bar(windata, 16.00, 13.00)

___
### Distribución de Resultados

In [ ]:
win_size = 50


In [ ]:
stock_df.index.get_loc('2017-01-26')

In [ ]:
def trade_distribution(stock_df, target_ret, loss_ret, win_size):
    results = {}
    durations = {}

    for idate in stock_df.index[:-(win_size+1)]:
        idx = stock_df.index.get_loc(idate)
        p_in = stock_df.close.loc[idate]
        windata = stock_df.iloc[idx + 1: idx+ win_size + 1]

        target = p_in * (1 + target_ret)
        loss = p_in * (1 - loss_ret)

        trade_date, p_out = out_trade_bar(windata, target, loss)
        results[idate] = p_out/p_in - 1
        durations[idate] = idx - stock_df.index.get_loc(trade_date)
    return pd.Series(results), pd.Series(durations)

In [ ]:
trade_rets, trade_durations = trade_distribution(stock_df=stock_df,
                                                 target_ret = 0.06,
                                                 loss_ret = 0.03,
                                                 win_size=30)

In [ ]:
print(trade_rets.mean())
trade_rets.hist()

___
Exploramos la media de resultados sobre un espacio de parámetros

In [ ]:
target_space = list(range(2, 15, 2))
loss_space = list(range(2, 15, 2))
win_size = 30
res_matrix = pd.DataFrame(0, index=target_space, columns=loss_space)
dur_matrix = pd.DataFrame(0, index=target_space, columns=loss_space)
for itarget in target_space:
    print(f'computing target {itarget}...')
    for jloss in loss_space:
        rets, durs = trade_distribution(stock_df=stock_df,
                                        target_ret = itarget/100.0,
                                        loss_ret = jloss/100.0,
                                        win_size=win_size)
        res_matrix.loc[itarget, jloss] = rets.mean()
        dur_matrix.loc[itarget, jloss] = durs.mean()


        

In [ ]:
sns.heatmap(res_matrix.round(3), annot=True)

In [ ]:
sns.heatmap(dur_matrix.round(3), annot=True)

____
### Observaciones Finales
- Aquí hemos explorado el resultado sobre trades sobre todos los días posibles
- Tiene sentido ver el cambio sobre una selección de días, por ejemplo a partir de nuestras señales de entrada